In [1]:
import duckdb

# Connect to an in-memory DuckDB instance
con = duckdb.connect(database=':memory:')

# Took me some trial and error comparing it to 'wc -l' to understand that this seems to be
# the best way to reads these files
sponsor_times = con.read_csv(
    "sb-mirror/sponsorTimes.csv",
    header=True,
    columns={
        "videoID": "VARCHAR",
        "startTime": "DOUBLE",
        "endTime": "DOUBLE",
        "votes": "INTEGER",
        "locked": "INTEGER",
        "incorrectVotes": "INTEGER",
        "UUID": "VARCHAR",
        "userID": "VARCHAR",
        "timeSubmitted": "DOUBLE",
        "views": "INTEGER",
        "category": "VARCHAR",
        "actionType": "VARCHAR",
        "service": "VARCHAR",
        "videoDuration": "DOUBLE",
        "hidden": "INTEGER",
        "reputation": "DOUBLE",
        "shadowHidden": "INTEGER",
        "hashedVideoID": "VARCHAR",
        "userAgent": "VARCHAR",
        "description": "VARCHAR",
    },
    ignore_errors=True,
    quotechar="",
)


video_info = con.read_csv(
    "sb-mirror/videoInfo.csv",
    header=True,
    columns={
        "videoID": "VARCHAR",
        "channelID": "VARCHAR",
        "title": "VARCHAR",
        "published": "DOUBLE",
    },
    ignore_errors=True,
    quotechar=None,
)

# Ignore warnings
import warnings
warnings.filterwarnings('ignore')


In [2]:
con.sql('DESCRIBE SELECT * FROM sponsor_times')

┌────────────────┬─────────────┬─────────┬─────────┬─────────┬─────────┐
│  column_name   │ column_type │  null   │   key   │ default │  extra  │
│    varchar     │   varchar   │ varchar │ varchar │ varchar │ varchar │
├────────────────┼─────────────┼─────────┼─────────┼─────────┼─────────┤
│ videoID        │ VARCHAR     │ YES     │ NULL    │ NULL    │ NULL    │
│ startTime      │ DOUBLE      │ YES     │ NULL    │ NULL    │ NULL    │
│ endTime        │ DOUBLE      │ YES     │ NULL    │ NULL    │ NULL    │
│ votes          │ INTEGER     │ YES     │ NULL    │ NULL    │ NULL    │
│ locked         │ INTEGER     │ YES     │ NULL    │ NULL    │ NULL    │
│ incorrectVotes │ INTEGER     │ YES     │ NULL    │ NULL    │ NULL    │
│ UUID           │ VARCHAR     │ YES     │ NULL    │ NULL    │ NULL    │
│ userID         │ VARCHAR     │ YES     │ NULL    │ NULL    │ NULL    │
│ timeSubmitted  │ DOUBLE      │ YES     │ NULL    │ NULL    │ NULL    │
│ views          │ INTEGER     │ YES     │ NULL    

In [22]:
con.sql("SELECT videoID, startTime, endTime, videoDuration, votes, locked, category FROM sponsor_times WHERE category IN ('sponsor', 'selfpromo') ORDER BY RANDOM() LIMIT 5")

┌─────────────┬───────────┬───────────┬───────────────┬───────┬────────┬──────────┐
│   videoID   │ startTime │  endTime  │ videoDuration │ votes │ locked │ category │
│   varchar   │  double   │  double   │    double     │ int32 │ int32  │ varchar  │
├─────────────┼───────────┼───────────┼───────────────┼───────┼────────┼──────────┤
│ QRxo_fH8bMA │       0.0 │       2.7 │       526.941 │     1 │      0 │ sponsor  │
│ 3esg23Dsmmk │       0.0 │       0.0 │       194.221 │    -2 │      0 │ sponsor  │
│ qdTN4gvLygk │       0.0 │       5.0 │       292.861 │     0 │      0 │ sponsor  │
│ CSGvpfA1sew │ 46.249058 │ 55.613075 │           0.0 │     4 │      0 │ sponsor  │
│ DPuwedsAv2o │   1916.33 │  2151.799 │      7638.854 │     1 │      0 │ sponsor  │
└─────────────┴───────────┴───────────┴───────────────┴───────┴────────┴──────────┘

In [23]:
# Count 
con.sql("SELECT DISTINCT category FROM sponsor_times")

┌──────────────────┐
│     category     │
│     varchar      │
├──────────────────┤
│ outro            │
│ interaction      │
│ intro            │
│ preview          │
│ music_offtopic   │
│ moreCategories   │
│ sponsor          │
│ filler           │
│ selfpromo        │
│ exclusive_access │
│ chapter          │
│ poi_highlight    │
├──────────────────┤
│     12 rows      │
└──────────────────┘

In [24]:
# What does the video_info table looks like
con.sql("SELECT * FROM video_info LIMIT 5")

┌─────────────┬──────────────────────────┬──────────────────────────────────────────────────────────────────┬──────────────┐
│   videoID   │        channelID         │                              title                               │  published   │
│   varchar   │         varchar          │                             varchar                              │    double    │
├─────────────┼──────────────────────────┼──────────────────────────────────────────────────────────────────┼──────────────┤
│ QyTRvbb3gkk │ UCr7tNSNf7_aEEh5P-F5mE4A │ Kronii Got Friendzoned by Chat but She End Up Streamerzoned Them │ 1634428800.0 │
│ qU1Yv58EXcc │ UCo_IB5145EVNcf8hw1Kku7w │ Game Theory: Minecraft's DARKEST Timeline! (Hermitcraft SMP)     │ 1634428800.0 │
│ yKkVHBh9DQk │ UCXJkLU1wZVqZjjVe1MuRj-A │ TRIPLE RECORD EN GLOBILLOS? 🎈                                   │ 1633564800.0 │
│ 7wCZSBOX7eM │ UCg83RGdRpwfvoFEuE2zWKZA │ Johnny vs. Nickelodeon All-Star Brawl (Sponsored)                │ 1633392000.0 │
│

In [25]:
# How many distinct channels are there?
con.sql("SELECT COUNT(distinct channelID) as distinct_channels FROM video_info LIMIT 5")

┌───────────────────┐
│ distinct_channels │
│       int64       │
├───────────────────┤
│            969181 │
└───────────────────┘

In [26]:
# Select the top 5 channel with the most amount of video ids
con.sql("SELECT channelID, COUNT(videoID) as video_count FROM video_info GROUP BY channelID ORDER BY video_count DESC LIMIT 5")

┌──────────────────────────┬─────────────┐
│        channelID         │ video_count │
│         varchar          │    int64    │
├──────────────────────────┼─────────────┤
│ NULL                     │      701396 │
│ UC_gUM8rL-Lrg6O3adPW9K1g │       96858 │
│ UCx8Z14PpntdaxCt2hakbQLQ │       39425 │
│ UCeY0bbntWzzVIaj2z3QigXg │       35656 │
│ UCsT0YIqwnpJCM-mx7-gSA4Q │       25050 │
└──────────────────────────┴─────────────┘

In [27]:
# Video with the highest amount of ad time
# First let's check if there are no duplicates in sponsor_times table
con.sql("SELECT count(videoID) as video_count, count(distinct videoID) as distinct_video_count FROM sponsor_times").show()

# Okay, there are, problably because one video has multiple sponsor times, of course..
# Now let's sum up the ad time for each video, grouping by videoID
con.sql("SELECT videoID, SUM(endTime - startTime) as total_ad_time FROM sponsor_times GROUP BY videoID ORDER BY total_ad_time DESC LIMIT 5")

# Literally all of these were for livestreams that are not available anymore. Any way we can check if they are "normal" videos?
# Apparently not. We would have to see based on the video duration and guess if it's a livestream or not.

┌─────────────┬──────────────────────┐
│ video_count │ distinct_video_count │
│    int64    │        int64         │
├─────────────┼──────────────────────┤
│    16112634 │              7511579 │
└─────────────┴──────────────────────┘



┌─────────────┬───────────────┐
│   videoID   │ total_ad_time │
│   varchar   │    double     │
├─────────────┼───────────────┤
│ Q6QR4979KIQ │ 29269449.0616 │
│ mg7FweYjasE │    18057695.0 │
│ 1oWx-V2L5ic │    16027781.0 │
│ inkWP2Wazag │     7522424.5 │
│ h4ZnVSRlg9g │   3596431.912 │
└─────────────┴───────────────┘

In [28]:
# Let's get some videos that have duplicated sponsor_times entries, just to check them
con.sql("SELECT videoID, COUNT(videoID) as video_count FROM sponsor_times GROUP BY videoID ORDER BY video_count DESC LIMIT 5").show()

# Hmm, so many entries. Ofc they are long videos but still. Maybe it means that this table has entries for other stuff?
# Let's do the same but filtering for a video that has a shorter duration (10min)
con.sql("SELECT videoID, COUNT(videoID) as video_count FROM sponsor_times WHERE videoDuration < 600 GROUP BY videoID ORDER BY video_count DESC LIMIT 5").show()

┌─────────────┬─────────────┐
│   videoID   │ video_count │
│   varchar   │    int64    │
├─────────────┼─────────────┤
│ "           │        3506 │
│ SYd5F4gIH90 │         886 │
│ ESC2oMp_RVE │         863 │
│ uyp7UvBPsjc │         838 │
│ hM101bVkyoo │         838 │
└─────────────┴─────────────┘

┌─────────────┬─────────────┐
│   videoID   │ video_count │
│   varchar   │    int64    │
├─────────────┼─────────────┤
│ "           │        3506 │
│ mIB389tqzCI │         492 │
│ Bn24gXSL4B0 │         463 │
│ 0hMdQAjy43A │         381 │
│ fKzVK1Di3Dw │         379 │
└─────────────┴─────────────┘



In [29]:
# Okay, so maybe the entries are not only for a single ad space. It's for each person that voted?! The first video is a LTT video with only 15 minutes of duration.
# Let's get 10 lines from this video just to see what they look like
con.sql("SELECT * FROM sponsor_times WHERE videoID = 'mIB389tqzCI' LIMIT 10").show()

┌─────────────┬───────────┬───────────┬───────┬────────┬────────────────┬──────────────────────────────────────────────────────────────────┬──────────────────────────────────────────────────────────────────┬─────────────────┬───────┬──────────┬────────────┬─────────┬───────────────┬────────┬────────────┬──────────────┬──────────────────────────────────────────────────────────────────┬───────────┬─────────────┐
│   videoID   │ startTime │  endTime  │ votes │ locked │ incorrectVotes │                               UUID                               │                              userID                              │  timeSubmitted  │ views │ category │ actionType │ service │ videoDuration │ hidden │ reputation │ shadowHidden │                          hashedVideoID                           │ userAgent │ description │
│   varchar   │  double   │  double   │ int32 │ int32  │     int32      │                             varchar                              │                             var

In [30]:
# Okay, there is a "locked" column. Probably means that it's the "finalized" times of the sponsors.
# Let's focus on this same video and get only when it's locked
con.sql("SELECT * FROM sponsor_times WHERE videoID = 'mIB389tqzCI' AND locked = 1").show()

# Okay, this is weird. This is an LTT video and it has a ad in the beginning and one in the end.
# The one in the beginning is exactly as it shows here, but the outro ad,
# which gets properly highlighted in my video player as a sponsor for Sponsorblock
# is not present here as locked... Weird.
con.sql("SELECT * FROM sponsor_times WHERE videoID = 'mIB389tqzCI' and endTime > 100 order by votes desc").show()

# There are a bunch of lines for ad segments that starts around the 14min mark, which is actually the second ad.
# But there are no locked, unfortunately. So I will need to come up with some logic, probably, to find all the actual, correct,
# ad segments of a video?!


┌─────────────┬───────────┬───────────┬───────┬────────┬────────────────┬──────────────────────────────────────────────────────────────────┬──────────────────────────────────────────────────────────────────┬─────────────────┬───────┬──────────┬────────────┬─────────┬───────────────┬────────┬────────────┬──────────────┬──────────────────────────────────────────────────────────────────┬───────────┬─────────────┐
│   videoID   │ startTime │  endTime  │ votes │ locked │ incorrectVotes │                               UUID                               │                              userID                              │  timeSubmitted  │ views │ category │ actionType │ service │ videoDuration │ hidden │ reputation │ shadowHidden │                          hashedVideoID                           │ userAgent │ description │
│   varchar   │  double   │  double   │ int32 │ int32  │     int32      │                             varchar                              │                             var

In [31]:
# Before doing that, let's dive a bit deeper.
# Let's get here all videos that have more than ONE locked line.
con.sql("SELECT videoID, COUNT(*) as locked_count FROM sponsor_times WHERE locked = 1 GROUP BY videoID HAVING COUNT(*) > 1 ORDER BY locked_count DESC").show()

┌─────────────┬──────────────┐
│   videoID   │ locked_count │
│   varchar   │    int64     │
├─────────────┼──────────────┤
│ oI5OyGlFHMc │          823 │
│ YwOXuyYW4so │          301 │
│ CBfGIR0NqCE │          284 │
│ 9OeH8uKnya0 │          278 │
│ KdZU1vhxkg8 │          250 │
│ WFyiv9C2-Yk │          198 │
│ E7MQb9Y4FAE │          182 │
│ JucppA13Wxk │          156 │
│ 2wunYAEkF7U │          146 │
│ Hqwviv1MKGQ │          144 │
│      ·      │            · │
│      ·      │            · │
│      ·      │            · │
│ ym47K3qBeyQ │            5 │
│ wWzg4wRtBkE │            5 │
│ YiuixhSB7BU │            5 │
│ 05K5glVCwis │            5 │
│ kvQToWGBoNk │            5 │
│ 2qGt129Vwjg │            5 │
│ 4FaQiM-Mex8 │            5 │
│ vauCU4b8c5E │            5 │
│ gL9SG81EUyM │            5 │
│ i2tt4qzHIQM │            5 │
├─────────────┴──────────────┤
│ ? rows           2 columns │
└────────────────────────────┘



In [32]:
# Wtf, there are videos with many locked counts. 
# I picked up here a random one, that happened to also be from LTT, that has 4 locked lines
# ID: RTTiQeXXrhI
# Let's focus on those locked lines:
con.sql("SELECT * FROM sponsor_times WHERE videoID = 'RTTiQeXXrhI' AND locked = 1").show()

# Oh wow, so one of them (55s to 64s) is actually the video INTRO. And we can even see in the 'category'. Crazy.

┌─────────────┬───────────┬───────────┬───────┬────────┬────────────────┬──────────────────────────────────────────────────────────────────┬──────────────────────────────────────────────────────────────────┬─────────────────┬───────┬───────────┬────────────┬─────────┬───────────────┬────────┬────────────┬──────────────┬──────────────────────────────────────────────────────────────────┬───────────┬─────────────┐
│   videoID   │ startTime │  endTime  │ votes │ locked │ incorrectVotes │                               UUID                               │                              userID                              │  timeSubmitted  │ views │ category  │ actionType │ service │ videoDuration │ hidden │ reputation │ shadowHidden │                          hashedVideoID                           │ userAgent │ description │
│   varchar   │  double   │  double   │ int32 │ int32  │     int32      │                             varchar                              │                             v

In [33]:
# Let's quickly print all categories around, and hmm, also the time they appear on locked lines.
con.sql("SELECT category, count(*) as count FROM sponsor_times WHERE locked = 1 GROUP BY category ORDER BY count(*) DESC").show()

# Okay, this is crazy, I had no clue about the outro/intro counting, as I don't have them enabled on my SponsorBlock extension.
# So from now on, I will only care about "sponsor" and "selfpromo".

┌──────────────────┬───────┐
│     category     │ count │
│     varchar      │ int64 │
├──────────────────┼───────┤
│ outro            │ 84518 │
│ intro            │ 84453 │
│ sponsor          │ 67508 │
│ selfpromo        │ 52234 │
│ interaction      │ 45523 │
│ filler           │ 38319 │
│ poi_highlight    │ 21235 │
│ chapter          │ 18986 │
│ music_offtopic   │ 15097 │
│ preview          │ 12956 │
│ exclusive_access │  1342 │
├──────────────────┴───────┤
│ 11 rows        2 columns │
└──────────────────────────┘



In [34]:
# Okay, go again. Let's find videos with many locked, but only sponsor or self promo.
# We are hoping that we don't have to come up with a logic to find the actual ad segments,
# and that we can simply look for locked=1 and category in ("sponsor", "selfpromo").
# Let's do that:
con.sql("SELECT videoID, COUNT(*) as locked_count FROM sponsor_times WHERE locked = 1 AND category IN ('sponsor', 'selfpromo') GROUP BY videoID").show()

# Hmm, of course I picked another random LTT video here. Ngy9TIbREJE
# This one shows three ads, which is correct.

# Okay, so like, beforehand we had found a video that I knew that there were more than one ad, but it only showed up as one in this table.
# So now I need to do it flipped. I will open random videos on YouTube and look for them here and see if they match.

┌─────────────┬──────────────┐
│   videoID   │ locked_count │
│   varchar   │    int64     │
├─────────────┼──────────────┤
│ HPPsMyy1HBc │            1 │
│ jP6rPqhrIJg │            1 │
│ NqswxL3RJuM │            1 │
│ nc0fJ36y6Ws │            1 │
│ N1kx_0DxPQo │            2 │
│ 77An0t_LRyc │            1 │
│ eVzKYLE2YgA │            1 │
│ t9rjBffVet0 │            2 │
│ jhHnShlmIiE │            1 │
│ MWRG9WAk9gU │            1 │
│      ·      │            · │
│      ·      │            · │
│      ·      │            · │
│ mBLdUFrdMaE │            2 │
│ hk-PTWIbYq8 │            1 │
│ dN6NXB5s1UU │            1 │
│ Gm0HaA5Mk5o │            2 │
│ zIjLT-hQ56A │            1 │
│ fn2Zk7xgUoA │            1 │
│ 7rtPjZVHxqo │            1 │
│ 2WKWcBeAkHc │            1 │
│ yegRHiaao7U │            1 │
│ P1x0oMGWDkw │            4 │
├─────────────┴──────────────┤
│ ? rows           2 columns │
└────────────────────────────┘



In [35]:
# Okay, here are 5 videos that I found here, from some different timeframes
# ZH2uEhLxNT4 --> 2 ads, one in the beginning and one in the end
# 6wgHq9NZru0 --> Same as above
# L8hTUcchAno --> One in the middle
# nC_dmiM3mMA --> One kinda in the beginning
# CxfFKuSfQ8c --> One in the end.

# 1st video:
con.sql("SELECT * FROM sponsor_times WHERE videoID = 'ZH2uEhLxNT4' AND locked = 1").show()

# 2nd video:
con.sql("SELECT * FROM sponsor_times WHERE videoID = '6wgHq9NZru0' AND locked = 1").show()

# 3rd video:
con.sql("SELECT * FROM sponsor_times WHERE videoID = 'L8hTUcchAno' AND locked = 1").show()

# 4th video:
con.sql("SELECT * FROM sponsor_times WHERE videoID = 'nC_dmiM3mMA' AND locked = 1").show()

# 5th video:
con.sql("SELECT * FROM sponsor_times WHERE videoID = 'CxfFKuSfQ8c' AND locked = 1").show()


# Hmm, some of them shows no locked lines. 

┌─────────┬───────────┬─────────┬───────┬────────┬────────────────┬─────────┬─────────┬───────────────┬───────┬──────────┬────────────┬─────────┬───────────────┬────────┬────────────┬──────────────┬───────────────┬───────────┬─────────────┐
│ videoID │ startTime │ endTime │ votes │ locked │ incorrectVotes │  UUID   │ userID  │ timeSubmitted │ views │ category │ actionType │ service │ videoDuration │ hidden │ reputation │ shadowHidden │ hashedVideoID │ userAgent │ description │
│ varchar │  double   │ double  │ int32 │ int32  │     int32      │ varchar │ varchar │    double     │ int32 │ varchar  │  varchar   │ varchar │    double     │ int32  │   double   │    int32     │    varchar    │  varchar  │   varchar   │
├─────────┴───────────┴─────────┴───────┴────────┴────────────────┴─────────┴─────────┴───────────────┴───────┴──────────┴────────────┴─────────┴───────────────┴────────┴────────────┴──────────────┴───────────────┴───────────┴─────────────┤
│                                   

In [36]:
# Just to make sure my dataset is correct, let's get the 1st and the last video and see if they are correct.
# 1st video:
con.sql("SELECT * FROM sponsor_times WHERE videoID = 'ZH2uEhLxNT4'").show()

# Last video:
con.sql("SELECT * FROM sponsor_times WHERE videoID = 'CxfFKuSfQ8c'").show()

# Yup, these are correct, but, yeah, no locked. Which means we will need to come up with some logic to find the actual ad segments.



┌─────────────┬───────────┬──────────┬───────┬────────┬────────────────┬───────────────────────────────────────────────────────────────────┬──────────────────────────────────────────────────────────────────┬─────────────────┬───────┬──────────┬────────────┬─────────┬───────────────┬────────┬────────────┬──────────────┬──────────────────────────────────────────────────────────────────┬─────────────────────────────────────────┬─────────────┐
│   videoID   │ startTime │ endTime  │ votes │ locked │ incorrectVotes │                               UUID                                │                              userID                              │  timeSubmitted  │ views │ category │ actionType │ service │ videoDuration │ hidden │ reputation │ shadowHidden │                          hashedVideoID                           │                userAgent                │ description │
│   varchar   │  double   │  double  │ int32 │ int32  │     int32      │                              varchar   

# Building a logic to nail down the ad segments.

In [37]:
# Let's focus on this video: 6wgHq9NZru0
con.sql("SELECT * FROM sponsor_times WHERE videoID = '6wgHq9NZru0' ORDER BY startTime asc").show()

┌─────────────┬───────────┬──────────┬───────┬────────┬────────────────┬───────────────────────────────────────────────────────────────────┬──────────────────────────────────────────────────────────────────┬─────────────────┬───────┬──────────┬────────────┬─────────┬───────────────┬────────┬────────────┬──────────────┬──────────────────────────────────────────────────────────────────┬──────────────────────────────────────────┬─────────────┐
│   videoID   │ startTime │ endTime  │ votes │ locked │ incorrectVotes │                               UUID                                │                              userID                              │  timeSubmitted  │ views │ category │ actionType │ service │ videoDuration │ hidden │ reputation │ shadowHidden │                          hashedVideoID                           │                userAgent                 │ description │
│   varchar   │  double   │  double  │ int32 │ int32  │     int32      │                              varchar 

In [ ]:
# To find if these rows are probably the same ad segment, we will do the following:
# 1. Order them by the start time
# 2. Compare one row with the previous.
# 3. If the start of the current row is greater than the end of the previous row, we will consider it a new ad segment.
# 4. We coalesce to False just to deal with the first row, as it will never be a new segment.
# 5. Then we create a new col, where we sum up the new_segment column, and add 1 to it. This will give us the ad segment number.
test_overlaps = con.sql("""
SELECT startTime, endTime, votes, locked,
       LAG(startTime) OVER (PARTITION BY videoID ORDER BY startTime) AS prev_start,
       LAG(endTime) OVER (PARTITION BY videoID ORDER BY startTime) AS prev_end,
       COALESCE(startTime > LAG(endTime) OVER (PARTITION BY videoID ORDER BY startTime), false) AS new_segment
FROM sponsor_times
WHERE videoID = '6wgHq9NZru0'
    AND category IN ('sponsor', 'selfpromo')
ORDER BY startTime
""")

test_overlaps.show()

# Now create an "ad_segment" column that will start at 1 and then increment every time overlaps is false
test_ad_segments = con.sql("""
SELECT
    *,
    SUM(new_segment) OVER (ORDER BY startTime) + 1 AS ad_segment
FROM test_overlaps
""")

test_ad_segments.show()

# Now we for each segment we can order them by locked and votes, making sure to either get the one locked, or the one with the most votes.
con.sql("""
SELECT DISTINCT ON (ad_segment)
        *
FROM test_ad_segments
ORDER BY ad_segment, locked DESC, votes DESC
""").show()


┌───────────┬──────────┬───────┬────────┬────────────┬──────────┬─────────────┐
│ startTime │ endTime  │ votes │ locked │ prev_start │ prev_end │ new_segment │
│  double   │  double  │ int32 │ int32  │   double   │  double  │   boolean   │
├───────────┼──────────┼───────┼────────┼────────────┼──────────┼─────────────┤
│    30.521 │     52.9 │     0 │      0 │       NULL │     NULL │ false       │
│     30.59 │    52.94 │    13 │      1 │     30.521 │     52.9 │ false       │
│    36.872 │   52.929 │    -2 │      0 │      30.59 │    52.94 │ false       │
│    959.53 │ 1010.753 │     8 │      1 │     36.872 │   52.929 │ true        │
└───────────┴──────────┴───────┴────────┴────────────┴──────────┴─────────────┘

┌───────────┬──────────┬───────┬────────┬────────────┬──────────┬─────────────┬────────────┐
│ startTime │ endTime  │ votes │ locked │ prev_start │ prev_end │ new_segment │ ad_segment │
│  double   │  double  │ int32 │ int32  │   double   │  double  │   boolean   │   int128   │


In [39]:
# Okay, now we're only missing applying this to the overall dataset, which will change the query around a bit.

new_segments = con.sql("""
SELECT
    COALESCE(startTime > LAG(endTime) OVER (PARTITION BY videoID ORDER BY startTime), false) AS new_ad_segment,
    *
FROM sponsor_times
WHERE category IN ('sponsor', 'selfpromo')
""")

ad_segments = con.sql("""
SELECT
    SUM(new_ad_segment) OVER (PARTITION BY videoID ORDER BY startTime) + 1 AS ad_segment,
    *
FROM new_segments
""")

# Now before we just get the "correct" ad segments, let's try for the same video we did above
con.sql("select * from ad_segments where videoID = '6wgHq9NZru0'").show()

# Looks good! Now let's just do the final query to get the best row for each ad segmenta and each video
unique_ad_segments = con.sql("""
SELECT DISTINCT ON (videoID, ad_segment)
    *
FROM ad_segments
ORDER BY videoID, ad_segment, locked DESC, votes DESC
""")

# Lets, again, try for the same video
con.sql("select * from unique_ad_segments where videoID = '6wgHq9NZru0'").show()


┌────────────┬────────────────┬─────────────┬───────────┬──────────┬───────┬────────┬────────────────┬───────────────────────────────────────────────────────────────────┬──────────────────────────────────────────────────────────────────┬─────────────────┬───────┬──────────┬────────────┬─────────┬───────────────┬────────┬────────────┬──────────────┬──────────────────────────────────────────────────────────────────┬──────────────────────────────────────────┬─────────────┐
│ ad_segment │ new_ad_segment │   videoID   │ startTime │ endTime  │ votes │ locked │ incorrectVotes │                               UUID                                │                              userID                              │  timeSubmitted  │ views │ category │ actionType │ service │ videoDuration │ hidden │ reputation │ shadowHidden │                          hashedVideoID                           │                userAgent                 │ description │
│   int128   │    boolean     │   varchar   │  dou

In [40]:
# Looks good enough to me! I think we're ready to start actually analyzing this data.